# Text search with Tries

— _Chidi Williams_


## 1. Find the prefix


Let's write a program that checks if a list of words contains a prefix:


In [ ]:
from typing import List
from pprint import pprint


def list_insert(dictionary: List[str], word: str) -> None:
    dictionary.append(word)


dictionary = []
list_insert(dictionary, 'ant')
list_insert(dictionary, 'antelope')
list_insert(dictionary, 'bear')
list_insert(dictionary, 'cat')
list_insert(dictionary, 'dog')

pprint(dictionary)


In [ ]:
def list_contains_prefix(dictionary: List[str], prefix: str) -> bool:
    for word in dictionary:
        if len(word) < len(prefix):
            continue
        prefixed = True
        for i, _ in enumerate(prefix):
            if prefix[i] != word[i]:
                prefixed = False
        if prefixed:
            return True
    return False


assert list_contains_prefix(dictionary, 'ant') is True
assert list_contains_prefix(dictionary, 'bear') is True
assert list_contains_prefix(dictionary, 'lion') is False


## 2. Find the prefix — Review


- To find the matches in `contains_prefix`, we check every word in the dictionary and then compare the prefix to the word.

- If the length of the dictionary is $p$ and the length of the prefix is $q$,
  the runtime complexity of finding the prefix is $O(p*q)$.

- **Can we do better?**


## 3. How to find a thing


- Think about how we buy things we need:

  - Want some food? Go to the food market!

    <img src="https://beta.techcrunch.com/wp-content/uploads/2016/12/3429168502_121cd39fa4_o.jpg?w=680" width=400 >

  - Want a car? Go to the car dealer!

    <img src="https://i1.wp.com/naijaxtreme.com/wp-content/uploads/2021/01/Car-Dealers-In-Nigeria-2.png?fit=650%2C365&ssl=1" width=400 >

  - Want some books? Go to the bookstore!

    <img src="https://booksellers.ng/image/cache/catalog/hab8-890x470.jpg" width=400 >

- **We find things faster by grouping "related entities" together!**


## 4. Grouping the dictionary


- We'll use a similar technique to speed up our dictionary.

- Instead of putting all the words in one big list, we'll group them by their first characters.

- All the words starting with 'a' in one list, all the words starting with 'b' in another list, and so on.


In [ ]:
import string


def group_insert(dictionary: List[List[str]], word: str) -> None:
    group_index = string.ascii_lowercase.index(word[0])
    dictionary[group_index].append(word)


grouped_dictionary = [[] for i in range(26)]
group_insert(grouped_dictionary, 'ant')
group_insert(grouped_dictionary, 'antelope')
group_insert(grouped_dictionary, 'bear')
group_insert(grouped_dictionary, 'lion')
group_insert(grouped_dictionary, 'rhino')

assert grouped_dictionary[0][0] == 'ant'
assert grouped_dictionary[0][1] == 'antelope'
assert grouped_dictionary[1][0] == 'bear'
assert grouped_dictionary[11][0] == 'lion'

pprint(grouped_dictionary)


- Now, instead of searching the entire dictionary, we only check the "child dictionary" starting with the same character as the prefix.


In [ ]:
def group_contains_prefix(dictionary: List[List[str]], prefix: str) -> bool:
    group_index = string.ascii_lowercase.index(prefix[0])
    return list_contains_prefix(dictionary[group_index], prefix)


assert group_contains_prefix(grouped_dictionary, 'ant') is True
assert group_contains_prefix(grouped_dictionary, 'bear') is True
assert group_contains_prefix(grouped_dictionary, 'zebra') is False


## 5. Grouping the dictionary — Review


- Instead of searching the entire dictionary, we now only check the "child dictionary" starting with the same character as the prefix.

- It's just like going directly to the bookstore to buy books!

- Assuming the words are evenly distributed among the child dictionaries, the time complexity will be $O(1 + (p / 26) * q)$

  - Check for the correct child dictionary based on the first character: $O(1)$

  - Check for the prefix in the child dictionary: $O((p / 26) * q)$

- $O(1 + (p / 26)*q)$ is better than $O(p * q)$!

- **Can we do even better?**


## 6. Grouping the dictionary (twice)

- We'll applying the grouping technique again, this time grouping words by their first **and second** characters.

- All the words starting with 'aa' in one list, all the words starting with 'ab' in another list, both inside the 'a' list, etc.


In [ ]:
def group_2_insert(dictionary: List[List[List[str]]], word: str) -> None:
    group1_index = string.ascii_lowercase.index(word[0])
    group2_index = string.ascii_lowercase.index(word[1])
    dictionary[group1_index][group2_index].append(word)


grouped_2_dictionary = [[[] for i in range(26)] for i in range(26)]
group_2_insert(grouped_2_dictionary, 'ant')
group_2_insert(grouped_2_dictionary, 'antelope')
group_2_insert(grouped_2_dictionary, 'bear')
group_2_insert(grouped_2_dictionary, 'boar')
group_2_insert(grouped_2_dictionary, 'lion')
group_2_insert(grouped_2_dictionary, 'rhino')

pprint(grouped_2_dictionary)


## 7. Grouping the dictionary (twice) — Oops...

- **Wait a second...**

- What about one-letter words?

In [ ]:
from dataclasses import dataclass


@dataclass
class Dictionary:
    children: list
    end_of_word = False


def group_2_insert(dictionary: Dictionary, word: str) -> None:
    group = dictionary.children[string.ascii_lowercase.index(word[0])]

    if len(word) == 1:
        group.end_of_word = True
        return

    group = group.children[string.ascii_lowercase.index(word[1])]
    if len(word) == 2:
        group.end_of_word = True
        return

    group.children.append(word)


grouped_2_dictionary = Dictionary(children=[
    Dictionary(children=[
        Dictionary(children=[]) for i in range(26)
    ]) for i in range(26)
])
group_2_insert(grouped_2_dictionary, 'a')
group_2_insert(grouped_2_dictionary, 'ant')
group_2_insert(grouped_2_dictionary, 'antelope')
group_2_insert(grouped_2_dictionary, 'bear')
group_2_insert(grouped_2_dictionary, 'boar')
group_2_insert(grouped_2_dictionary, 'lion')
group_2_insert(grouped_2_dictionary, 'rhino')

pprint(grouped_2_dictionary)
pprint(grouped_2_dictionary.children[0].end_of_word)


- Now, we only check the "child dictionary" starting with the same **first and second** character as the prefix.

- But if the prefix is less than two characters long, we simply check for the `end_of_word` token.


In [ ]:
def group_2_contains_prefix(dictionary: Dictionary, prefix: str) -> bool:
    group = dictionary.children[string.ascii_lowercase.index(prefix[0])]
    if len(prefix) == 1:
        return group.end_of_word

    group = group.children[string.ascii_lowercase.index(prefix[1])]
    return list_contains_prefix(group.children, prefix)


assert group_2_contains_prefix(grouped_2_dictionary, 'ant') is True
assert group_2_contains_prefix(grouped_2_dictionary, 'bear') is True
assert group_2_contains_prefix(grouped_2_dictionary, 'zebra') is False

assert group_2_contains_prefix(grouped_2_dictionary, 'a') is True
assert group_2_contains_prefix(grouped_2_dictionary, 'z') is False


## 8. Grouping the dictionary (twice) - Review

- Assuming the words are evenly distributed among the child dictionaries, the time complexity will be $O(2 + (p / (26 * 26)) * q)$

  - Check for the correct child dictionary based on the first and second characters: $O(2)$

  - Check for the prefix in the child dictionary: $O((p / (26 * 26)) * q)$

- $O(2 + (p / (26 * 26)) * q)$ is even better than $O(1 + (p / 26) * q)$!

- **How much better can it get?**


## 9. Trie

- Group by not just the first one or two characters, but by **all** the characters in the word

- When we add "zebra" to the dictionary, we'll create groups for words starting with "z", "ze", "zeb", "zebr", and "zebra"

- This is a trie!


In [40]:
from typing import Optional


@dataclass
class Trie:
    children: List[Optional['Trie']]
    end_of_word: bool

    def __init__(self, children=[None for i in range(26)], end_of_word=False) -> None:
        self.children = children
        self.end_of_word = end_of_word


def trie_insert(trie: Trie, word: str) -> None:
    current: Trie = trie

    for char in word:
        group_index = string.ascii_lowercase.index(char)
        if current.children[group_index] is None:
            current.children[group_index] = Trie()

        current = current.children[group_index]

    current.end_of_word = True


trie = Trie()
trie_insert(trie, 'a')
trie_insert(trie, 'ant')
trie_insert(trie, 'antelope')
trie_insert(trie, 'bear')
trie_insert(trie, 'boar')
trie_insert(trie, 'lion')
trie_insert(trie, 'rhino')

pprint(trie)


Trie(children=[Trie(children=...,
                    end_of_word=True),
               Trie(children=...,
                    end_of_word=False),
               None,
               None,
               Trie(children=...,
                    end_of_word=True),
               None,
               None,
               Trie(children=...,
                    end_of_word=False),
               Trie(children=...,
                    end_of_word=False),
               None,
               None,
               Trie(children=...,
                    end_of_word=False),
               None,
               Trie(children=...,
                    end_of_word=True),
               Trie(children=...,
                    end_of_word=True),
               Trie(children=...,
                    end_of_word=False),
               None,
               Trie(children=...,
                    end_of_word=True),
               None,
               Trie(children=...,
                    end_of_word=True),
   

- That's not very easy to read. Let's add a pretty printer!

In [ ]:
def print_trie(trie: Trie, depth=0) -> None:
    for (i, child) in enumerate(trie.children):
        if child is not None:
            print(
                f'{"-": >{depth * 4}} {string.ascii_lowercase[i]} {"(word)" if child.end_of_word else ""}')
            print_trie(child, depth + 1)


print_trie(trie)


- **A trie is a _really_ compressed list of strings**


- We still need a way to check that the dictionary contains a prefix as before:

In [ ]:
def trie_contains_prefix(trie: Trie, prefix: str) -> bool:
    current: Trie = trie

    for char in prefix:
        group_index = string.ascii_lowercase.index(char)
        if current.children[group_index] is None:
            return False
        current = current.children[group_index]

    return True


assert trie_contains_prefix(trie, 'a') is True
assert trie_contains_prefix(trie, 'ant') is True
assert trie_contains_prefix(trie, 'be') is True
assert trie_contains_prefix(trie, 'bear') is True
assert trie_contains_prefix(trie, 'y') is False
assert trie_contains_prefix(trie, 'zeb') is False


## 10. Trie — Review

- The runtime complexity of finding the prefix in the trie is $O(q)$, where $q$ is the length of the prefix

- Much better than where we started, $O(p * q)$, where $p$ is the number of items in the dictionary

- We've moved all the work _up-front_: retrieving from the dictionary is now a lot faster at the cost of slower insertion

  - This is fine! It's fair to assume that we retrieve from dicionaries much more frequently than we add new words

  - _Trie_ comes from re*triev*al!

- Applications

  - Autocomplete

  - Text matching

  - Spell checking


## 11. Exercises

1. What is the time complexity of inserting a new word into the trie?

2. Extend the trie implementation to store and retrieve words containing uppercase letters and numbers.

3. Implement a function `trie_starts_with` that returns all the words in the trie that start with a given prefix.

   ```python
   trie = Trie()

   trie_insert(trie, 'ant')
   trie_insert(trie, 'antelope')
   trie_insert(trie, 'bear')
   trie_insert(trie, 'boar')
   trie_insert(trie, 'bee')
   trie_insert(trie, 'lion')

   trie_starts_with(trie, 'ant') # ['ant', 'antelope']
   trie_starts_with(trie, 'b')   # ['bear', 'boar', 'bee']
   trie_starts_with(trie, 'rh')  # []
   ```

4. Implement a function `trie_delete` that deletes a given word from a trie. The function should return `True` if the word was found and deleted.

   ```python
   trie = Trie()

   trie_insert(trie, 'ant')
   trie_insert(trie, 'antelope')

   assert trie_contains_prefix(trie, 'ante') is True

   assert trie_delete(trie, 'antelope') is True

   assert trie_contains_prefix(trie, 'ante') is False

   assert trie_delete(trie, 'gorilla') is False
   ```

5. Tries are useful for finding words that begin with a prefix, but they can also be constructed to return the substrings within a larger string. Implement `str_contains` using a trie.

   ```python
   assert str_contains('entrepreneurship', 'rep') is True
   assert str_contains('entrepreneurship', 'slow') is False
   assert str_contains('entrepreneurship', 'ship') is True
   ```
